# RAII and smart pointers

In the last section, we have discussed the powerful RAII idiom, which reduces the risk of improperly managed resources. Applied to the concept of memory management, RAII enables us to encapsulate new and delete calls within a class and thus present the programmer with a clean interface to the resource he intends to use. Since C++11, there exists a language feature called smart pointers, which builds on the concept of RAII and - without exaggeration - revolutionizes the way we use resources on the heap. Let’s take a look.

# 1. The Unique pointer

In [1]:
#include<iostream>
class MyClass
{
private:
    std::string _text;

public:
    MyClass() {}
    MyClass(std::string text) { _text = text; }
    ~MyClass() { std::cout << _text << " destroyed" << std::endl; }
    void setText(std::string text) { _text = text; }
};

In [2]:
void run_unique()
{
    // create unique pointer to proprietary class
    std::unique_ptr<MyClass> myClass1(new MyClass());
    std::unique_ptr<MyClass> myClass2(new MyClass("String 2"));
    std::unique_ptr<MyClass> myClass3 =  std::make_unique<MyClass>("String 3");

    // call member function using ->
    myClass1->setText("String 1");

    // use the dereference operator * 
    *myClass1 = *myClass2;

    // use the .get() function to retrieve a raw pointer to the object
    std::cout << "Objects have stack addresses " << myClass1.get() << " and " << myClass2.get() << std::endl;
}


In [3]:
run_unique()

Objects have stack addresses 0x556e2718ad10 and 0x556e2738d9c0
String 3 destroyed
String 2 destroyed
String 2 destroyed


# 2. The Shared Pointer

In [4]:
void run_shared()
{
    std::shared_ptr<MyClass> shared(new MyClass);
    std::cout << "shared pointer count = " << shared.use_count() << std::endl;
    
    // reallocate data to shared ptr
    shared.reset(new MyClass);
    std::cout << "shared pointer count = " << shared.use_count() << std::endl;
}

In [5]:
run_shared()

shared pointer count = 1
 destroyed
shared pointer count = 1
 destroyed


# 3. The Weak Pointer

In [6]:
void run_weak_counter()
{
    std::shared_ptr<int> mySharedPtr(new int);
    std::cout << "shared pointer count = " << mySharedPtr.use_count() << std::endl;

    std::weak_ptr<int> myWeakPtr1(mySharedPtr);
    std::weak_ptr<int> myWeakPtr2(myWeakPtr1);
    std::cout << "shared pointer count = " << mySharedPtr.use_count() << std::endl;
}

In [7]:
run_weak_counter();

shared pointer count = 1
shared pointer count = 1


In [8]:
void run_weak_expired()
{
    std::shared_ptr<int> mySharedPtr(new int);
    std::weak_ptr<int> myWeakPtr(mySharedPtr);

    mySharedPtr.reset(new int);

    if (myWeakPtr.expired() == true)
    {
        std::cout << "Weak pointer expired!" << std::endl;
    }
}

In [9]:
run_weak_expired()

Weak pointer expired!


# Converting between smart pointers

In [10]:
void run_convert_ptrs()
{
    // construct a unique pointer
    std::unique_ptr<MyClass> uniquePtr(new MyClass);
    
    // (1) shared pointer from unique pointer
    std::shared_ptr<MyClass> sharedPtr1 = std::move(uniquePtr);

    // (2) shared pointer from weak pointer
    std::weak_ptr<MyClass> weakPtr(sharedPtr1);
    std::shared_ptr<MyClass> sharedPtr2 = weakPtr.lock();
}

In [11]:
run_convert_ptrs()

 destroyed
